# Import libraries

In [1]:
import subprocess
import os
from helpers import parse_variables
import pandas as pd
import numpy as np

# Load simulation parameters

In [8]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
if 'k' not in globals():
    k = int(dict['k'])
    
if 'M' not in globals():
    M = float(dict['M'])
HWE = int(dict['HWE'])

nr_humans = int(dict['nr_humans'])
nr_snps = int(dict['nr_snps'])
bottleneck_nr = int(dict['bottleneck_nr'])

# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k

very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

complete = pd.concat([common, rare, very_rare], axis=1)
complete = ((complete*2)-1)

In [9]:
path_bottle = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/abyss_bottleneck"
bottle_file = [f for f in os.listdir(path_bottle) if int(f.split("_")[2]) ==  bottleneck_nr][0]
elapsed_time_bottleneck = float(bottle_file.split('_')[3].split('seconds')[0])
bottle = pd.read_pickle(f"{path_bottle}/{bottle_file}")

In [10]:
bottle

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim56,dim57,dim58,dim59,dim60,dim61,dim62,dim63,dim64,cluster
0,-0.192958,0.472057,-0.461037,-0.141970,0.523462,0.037412,-0.256095,0.044310,-0.383002,-0.064207,...,-0.158250,-0.002098,1.220213,-0.111987,0.456136,0.969642,-0.506950,0.025234,0.368444,0
1,-0.130909,0.914749,-0.190005,-0.479350,0.360581,-0.383142,-0.513258,-0.389784,-0.820086,-0.303477,...,-0.543561,0.045897,0.250448,-0.393348,-0.321920,0.474686,0.011526,-0.344862,0.520947,0
2,-0.472195,0.022355,0.640547,0.029756,-0.425412,-0.403838,0.574991,-0.404902,0.311971,-0.094813,...,0.389514,-0.069590,0.051776,-0.283522,0.160540,-0.340484,0.216087,-0.095389,0.200439,0
3,-0.177178,-0.091966,-0.409485,0.485531,-0.325592,-0.334142,0.212488,-0.076221,-0.280445,-0.422732,...,0.664186,0.342589,0.160483,-0.322925,-0.132099,0.006561,0.381173,0.039318,-0.266704,0
4,-0.120214,0.500082,0.174102,0.330117,-0.287405,-0.461805,-0.421212,0.146431,-0.860431,-0.214309,...,-0.042570,-0.009755,0.746475,-0.277062,-0.294182,1.177967,0.297135,-0.484517,0.272014,0
5,-0.089847,-0.201890,-0.476259,-0.474117,0.283553,-0.116755,-0.204721,0.196978,-0.536603,0.424283,...,-0.407263,0.336997,-0.098511,-0.195942,0.502124,0.436787,0.754011,-0.152693,0.350947,0
6,-0.509991,0.252306,-0.221619,-0.084988,-0.341667,-0.092986,-0.221672,0.259164,-0.635761,-0.428539,...,-0.295425,0.836006,0.790329,0.681859,-0.430761,0.108274,1.017860,-0.432513,0.376971,0
7,-0.394911,0.119993,0.057816,-0.473002,-0.208327,0.278308,-0.284074,-0.159384,0.269430,-0.072574,...,-0.119756,0.822243,-0.160333,0.036949,0.036111,0.047562,-0.629599,-0.378563,0.188312,0
8,-0.714121,0.608074,0.105764,-0.538820,0.082724,0.256538,0.280223,-0.132605,0.641037,0.454975,...,-0.446947,0.397067,-0.208512,0.152078,0.174529,-0.258701,-0.248567,0.304960,0.407646,0
9,-0.300733,0.076824,-0.206001,0.046688,-0.170695,0.371607,-0.413581,-0.386631,-0.752787,-0.482448,...,-0.547256,-0.203359,0.902615,0.151464,-0.698458,0.121663,0.339684,-0.259370,1.178084,0


In [11]:
path_pops_estimated_lds = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks_estimated_mafs/"

In [12]:
L*G

100

In [13]:
q2s_pop = []
twopqs_pop = []
p2s_pop = []

time_q2 = 0.0
time_p2 = 0.0
time_2pq = 0.0

for pop in os.listdir(path_pops_estimated_lds):
    print(pop)
    bottle_index = bottle[bottle['cluster']==int(pop)]
    path_estimated_lds = path_pops_estimated_lds + "/" + pop
    q2_files = [f for f in os.listdir(path_estimated_lds) if f.split(f"_")[6] == 'q2']
    q2_files = sorted(q2_files, key=lambda x: int(x.split('_')[0]))
    p2_files = [f for f in os.listdir(path_estimated_lds) if f.split(f"_")[6] == 'p2']
    p2_files = sorted(p2_files, key=lambda x: int(x.split('_')[0]))
    
    twopq_files = [f for f in os.listdir(path_estimated_lds) if f.split(f"_")[6] == '2pq']
    twopq_files = sorted(twopq_files, key=lambda x: int(x.split('_')[0]))
    
    
    q2s = []
    for q2_file in q2_files:
        time_q2 += float(q2_file.split('_pop_')[1].split("seconds")[0])
        path_q2_file = path_estimated_lds + '/' + q2_file
        q2 = pd.read_pickle(path_q2_file)
        q2s.append(q2)
    
    q2s = pd.concat(q2s, axis=1)
    q2s = q2s[list(complete.columns)]
    q2s_pop.append(q2s)

    p2s = []
    for p2_file in p2_files:
        time_p2 += float(p2_file.split('_pop_')[1].split("seconds")[0])

        path_p2_file = path_estimated_lds + '/' + p2_file
        p2 = pd.read_pickle(path_p2_file)
        p2s.append(p2)

    p2s = pd.concat(p2s, axis=1)
    p2s = p2s[list(complete.columns)]
    p2s_pop.append(p2s)

    
    twopqs = []
    for twopq_file in twopq_files:
        time_2pq += float(twopq_file.split('_pop_')[1].split("seconds")[0])
        path_2pq_file = path_estimated_lds + '/' + twopq_file
        twopq = pd.read_pickle(path_2pq_file)
        twopqs.append(twopq)

    twopqs = pd.concat(twopqs, axis=1)
    twopqs = twopqs[list(complete.columns)]
    twopqs_pop.append(twopqs)


q2s = pd.concat(q2s_pop, axis=0)
q2s = q2s.sort_index()

p2s = pd.concat(p2s_pop, axis=0)
p2s = p2s.sort_index()

twopqs = pd.concat(twopqs_pop, axis=0)
twopqs = twopqs.sort_index()

0


In [14]:
path_output = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/"

In [15]:
os.system(f"rm -rf {path_output}/estimated*")

0

In [16]:
q2s.to_pickle(f"{path_output}/estimated_q2s_via_esti_pop_{time_q2}seconds.pkl")
p2s.to_pickle(f"{path_output}/estimated_p2s_via_esti_pop_{time_p2}seconds.pkl")
twopqs.to_pickle(f"{path_output}/estimated_2pqs_via_esti_pop_{time_2pq}seconds.pkl")